## Implementation Details SoPhie

1. Feature Extractor Module
2. Attention Module
3. LSTM based GAN Module
4. Loss
5. Optimizer

# 1. Feature Extractor Module

### 1.1 Visual Features
- $V_{Ph}^t = CNN(I^t; W_{cnn})$
- Fine tuned on scene ground truth?
- Take ResNet features first, without pretraining.

### 1.2 Past Trajectory Features
- $V_{en}^t(i)= LSTM_{en}(X_i^t, h_{en}^t(i), W_{en})$
- $h_{en}^t(i)$ hidden state of the encoder at time $t$ for the agent $i$
- $V_{So}^t(i)=(V_{en}^t(\pi_j)-V_{en}^t(i) \vert \forall \pi_j \in [N]\i))$
- $\pi_j$ distance of closest agent


# 2. Attention Module

### 2.1 Physical
- $C_{Ph}^t(i)=ATT_{Ph}(V_{Ph}, h_{dec}^t(i); W_{Ph}$) , physical

### 2.2 Social
- $C_{So}^t(i)=ATT_{So}(V_{So}^t(i), h_{dec}^t(i); W_{So}$) , social

# 3. LSTM based GAN Module
### 3.1 Generator
- Generator input $C_G^t(i) = [C_{So}^t(i), C_{Ph}^t(i), z]$
- Generator output $\hat{Y}^\tau_i = LSTM_{dev}(C_G^t(i), h_{dec}^\tau(i); W_{dec})$

### 3.2 Discriminator
- Discriminator input $T_i^{1:\tau} \sim p(\hat{Y}^{1:\tau}_i, Y^{1:\tau}_i)$ , all states up to $\tau$
- Discriminator output $\hat{L}_i^\tau = LSTM_{dis}(T_i^\tau, h_{dis}^\tau(i); W_{dis})$

## 4. Loss
$L_{GAN}(\hat{L}_i^\tau, L_i^\tau)=\min_G \max_D E_{T_i^{1:\tau} \sim p(Y^{1:\tau}_i)} [L_i^\tau \log \hat{L}_i^\tau]+ E_{T_i^{1:\tau} \sim p(\hat{Y}^{1:\tau}_i)}[(1-L_i^\tau)\log(1-\hat{L}_i^\tau)]$

$L_{L2}(\hat{Y}^{\tau}_i, Y^{\tau}_i)$

$W* = \text{argmin}_W E_{i,\tau}[L_{GAN}+ \lambda L_{L2}]$

# 5. Optimizer
- Adam optimizer
- n_batch = 64
- lr = 0.001
- n_epochs = 200
- rotation and flipping
- coordinate normalization

### social attention
- emlp_dim = 16  #encoder

### gan
- ghid_dim = 32 #generate
- dhid_dim = 64 #discriminator
- dmlp_dim = 16 #decoder generator

### attention
- 4 mlp layers of sizes (64, 128, 64, 1) with relu, final softmax
- n_max = 32 else 0 # considered agents

### physical attention
- vgg 512 channels mlp to 16

### training
t_obs = 8
t_pred = 12
lambda = 1
- flipping, rotating scene
- coordinate normalization
- 


In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torchvision import models
from pathlib import Path
import cv2
import numpy as np
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
root_path = Path('/mnt/Clouds/MultimodalSophie/')

class Experiment:
    
    def __init__(self):
        super(Experiment, self).__init__()
        self.file_name = ''
        self.file_path = ''
        self.test_dir = ''
        self.train_dir = ''
        self.val_dir = ''
        
class ETH(Experiment):

    def __init__(self):
        super(ETH, self).__init__()
        self.root_path = root_path / 'data/eth/'
        self.video_file = self.root_path / 'seq_eth.avi'
        self.trajectory_file = self.root_path / 'test/biwi_eth.txt'
        self.H = np.array([[2.8128700e-02,   2.0091900e-03,  -4.6693600e+00], 
                           [8.0625700e-04,   2.5195500e-02,  -5.0608800e+00],
                           [3.4555400e-04,   9.2512200e-05,   4.6255300e-01]])

class Univ(Experiment):
    
    def __init__(self):
        super(Univ, self).__init__()
        self.root_path = root_path / 'data/univ/'
        self.video_file = self.root_path / 'students003.avi'
        self.trajectory_file = self.root_path / 'test/students003.txt'
        self.H = np.array([[  0.02104651,   0.        , -10.01813922],
                           [  0.        ,   0.02386598,  -2.79231966],
                           [  0.        ,   0.        ,   1.        ]])
    
class Hotel(Experiment):
    NotImplemented    
class Zara1(Experiment):
    NotImplemented
class Zara2(Experiment):
    NotImplemented
    

In [3]:
eth = ETH()

In [4]:
class Videodata:
    
    def __init__(self, experiment):
        self.file_path = experiment.file_path
        self.file_name = experiment.file_name
        self.homography = experiment.H
        self.video = cv2.VideoCapture(str(experiment.video_file))
        self.frame_width = int(self.video.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.frame_height = int(self.video.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.frame_count = int(self.video.get(cv2.CAP_PROP_FRAME_COUNT))
        
    def __getitem__(self, key):
        return NotImplemented
    
    def __len__(self):
        return NotImplemented
    
    def read_file(self, _path, delim='\t'):
        data = []
        if delim == 'tab':
            delim = '\t'
        elif delim == 'space':
            delim = ' '
        with open(_path, 'r') as f:
            for line in f:
                line = line.strip().split(delim)
                line = [float(i) for i in line]
                data.append(line)
        return np.asarray(data)
    
    def camcoordinates(self, xy):
        """Transform the meter coordinates with the homography matrix"""
        coords = xy.reshape(1, 1, -1)
        return cv2.perspectiveTransform(coords,  np.linalg.inv(self.homography)).squeeze()[::-1]
    
    def getFrame(self, fid):
        self.video.set(cv2.CAP_PROP_POS_FRAMES,fid)
        return self.video.read()[1]
    
    def staticImage(self):
        ret = True
        image = np.zeros((self.frame_height, self.frame_width, 3))
        while(ret):
            ret, img = self.video.read()
            if not ret:
                break
            image += img
        image /= self.frame_count
        image = image.astype('uint8')
        return image
        
    

In [5]:
V = Videodata(eth)

In [ ]:
x = V.staticImage()

In [ ]:
V.getFrame(100)

In [ ]:
traj = V.read_file('/mnt/Clouds/MultimodalSophie/sgan/datasets/eth/test/biwi_eth.txt')

In [ ]:
traj[np.where(traj[:, 1]==25)]

In [ ]:
for t in traj[np.where(traj[:, 1]==27)]:
    frame, aid, x, y = t
    fig, ax = plt.subplots(1, 1, figsize = [20, 10])
    ax.imshow(V.getFrame(frame))
    ax.plot(*V.camcoordinates(np.array([x, y])), 'r.', markersize = 30)
    plt.show()

In [ ]:
transform = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.ToTensor()])

In [ ]:
img = transform(Image.fromarray(x))

### Features

In [ ]:
resnet = models.resnet18(pretrained = True)
reassign_layers(resnet)
resnet.forward = _forward.__get__(resnet, resnet.__class__)
resnet.extract_layers = ['layer9']
feat_resnet = resnet(img.unsqueeze(0))['layer9'].squeeze().data.numpy()
plt.imshow(feat_resnet[50])

In [ ]:
vgg = models.vgg16(pretrained = True)
reassign_layers(vgg)
vgg.forward = _forward.__get__(vgg, vgg.__class__)
vgg.extract_layers = ['layer13']
feat_vgg = vgg(img.unsqueeze(0))['layer13'].squeeze().data.numpy()
plt.imshow(feat_vgg[50])

In [ ]:
resnet = models.resnet18(pretrained = True)

In [ ]:
class ResNetFeatures(nn.Module):

    def __init__(self):
        super(ResNetFeatures, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        
    def forward(self, img):
        if len(img.shape) == 3:
            img = img.unsqueeze(0)
        img = self.resnet.conv1(img)
        img = self.resnet.bn1(img)
        img = self.resnet.relu(img)
        img = self.resnet.maxpool(img)
        img = self.resnet.layer1(img)
        img = self.resnet.layer2(img)
        img = self.resnet.layer3(img)
        img = self.resnet.layer4(img)
        return img
    
class VGGFeatures(nn.Module):
    
    def __init__(self):
        super(VGGFeatures, self).__init__()
        self.vgg = models.vgg16(pretrained=True)
        
    def forward(self, img):
        if len(img.shape) == 3:
            img = img.unsqueeze(0)
        return self.vgg.features(img)

In [ ]:
resnet = ResNetFeatures(); vgg = VGGFeatures()
resfeats = resnet(img).data.numpy().squeeze()
vggfeats = vgg(img).data.numpy().squeeze()

### Dataset

In [104]:
class TrajectoryDataset(Dataset):

    def __init__(self, experiment, self.slen):
        super(TrajectoryDataset, self).__init__()
        self.video = Videodata(experiment)
        self.tdata = read_file(experiment.trajectory_file)
        self.frames = np.unique(self.tdata[:, 0])
        self.agents = np.unique(self.tdata[:, 1])
        
    def __len__(self):
        return NotImplemented
    
    def __getitem__(self, key):
        return NotImplemented

In [105]:
td = TrajectoryDataset(ETH())

In [130]:
[td.tdata[np.where(td.tdata[:, 0]==x)] for x in td.frames]

[array([[780.  ,   1.  ,   8.46,   3.59]]),
 array([[790.  ,   1.  ,   9.57,   3.79]]),
 array([[800.  ,   1.  ,  10.67,   3.99],
        [800.  ,   2.  ,  13.64,   5.8 ]]),
 array([[810.  ,   1.  ,  11.73,   4.32],
        [810.  ,   2.  ,  12.09,   5.75]]),
 array([[820.  ,   1.  ,  12.81,   4.61],
        [820.  ,   2.  ,  11.37,   5.8 ]]),
 array([[830.  ,   2.  ,  10.31,   5.97],
        [830.  ,   3.  ,  12.49,   6.6 ]]),
 array([[840.  ,   2.  ,   9.57,   6.24],
        [840.  ,   3.  ,  11.94,   6.77]]),
 array([[850.  ,   2.  ,   8.73,   6.34],
        [850.  ,   3.  ,  11.03,   6.84],
        [850.  ,   4.  ,  -1.32,   5.11],
        [850.  ,   5.  ,  -1.48,   4.43],
        [850.  ,   6.  ,  11.84,   5.82]]),
 array([[ 8.600e+02,  2.000e+00,  7.940e+00,  6.500e+00],
        [ 8.600e+02,  3.000e+00,  1.021e+01,  6.810e+00],
        [ 8.600e+02,  4.000e+00, -3.300e-01,  5.060e+00],
        [ 8.600e+02,  5.000e+00, -4.900e-01,  4.380e+00],
        [ 8.600e+02,  6.000e+00,  1.09

In [131]:
[td.tdata[np.where(td.tdata[:, 1]==x)] for x in td.agents]

[array([[780.  ,   1.  ,   8.46,   3.59],
        [790.  ,   1.  ,   9.57,   3.79],
        [800.  ,   1.  ,  10.67,   3.99],
        [810.  ,   1.  ,  11.73,   4.32],
        [820.  ,   1.  ,  12.81,   4.61]]),
 array([[ 8.000e+02,  2.000e+00,  1.364e+01,  5.800e+00],
        [ 8.100e+02,  2.000e+00,  1.209e+01,  5.750e+00],
        [ 8.200e+02,  2.000e+00,  1.137e+01,  5.800e+00],
        [ 8.300e+02,  2.000e+00,  1.031e+01,  5.970e+00],
        [ 8.400e+02,  2.000e+00,  9.570e+00,  6.240e+00],
        [ 8.500e+02,  2.000e+00,  8.730e+00,  6.340e+00],
        [ 8.600e+02,  2.000e+00,  7.940e+00,  6.500e+00],
        [ 8.700e+02,  2.000e+00,  7.170e+00,  6.620e+00],
        [ 8.800e+02,  2.000e+00,  6.470e+00,  6.680e+00],
        [ 8.900e+02,  2.000e+00,  5.860e+00,  6.820e+00],
        [ 9.000e+02,  2.000e+00,  5.240e+00,  6.980e+00],
        [ 9.100e+02,  2.000e+00,  4.870e+00,  7.160e+00],
        [ 9.200e+02,  2.000e+00,  4.510e+00,  7.580e+00],
        [ 9.300e+02,  2.000e+00,  

In [128]:
len(td.frames)

876

In [117]:
np.argmax([len(x) for x in a])

165

In [147]:
t = TrajectoryDataset(ETH().root_path / 'test')

In [191]:
t[0][1]

tensor([[[ 4.8700,  4.5100,  4.2000,  3.9500,  3.4700,  2.8200,  2.0100,
           1.2800,  0.5400, -0.1800, -0.8300, -1.5200],
         [ 7.1600,  7.5800,  7.3000,  7.7100,  7.8600,  8.0000,  8.0000,
           7.8200,  7.4000,  7.0600,  6.4300,  6.0500]],

        [[ 6.2900,  5.6200,  5.0600,  4.6900,  4.3500,  3.7600,  3.1900,
           2.6200,  1.7800,  1.0100,  0.0700, -0.7200],
         [ 7.0000,  7.1000,  7.0400,  7.0000,  7.0100,  6.9900,  6.8900,
           7.1300,  7.1500,  6.9600,  6.9100,  6.6600]]])

In [200]:
def getorder():
    pass
t[0][1].transpose(0, 2)[0].transpose(0, 1)[0]-t[0][1].transpose(0, 2)[0].transpose(0, 1)[1]

tensor([-1.4200,  0.1600])

In [187]:
torch.Tensor.transpose??

In [ ]:
torch.Tensor().transpose

In [177]:
[t.seq_list[0][0][i] - t.seq_list[0][1][i] for i in [0, 1]]

[array([-2.18, -2.37, -2.3 , -2.27, -2.19, -2.12, -1.92, -1.72, -1.42,
        -1.11, -0.86, -0.74, -0.88, -0.94, -1.18, -1.34, -1.24, -1.19,
        -0.9 , -0.8 ]),
 array([-0.63, -0.53, -0.5 , -0.31, -0.23, -0.17, -0.02,  0.14,  0.16,
         0.48,  0.26,  0.71,  0.85,  1.01,  1.11,  0.69,  0.25,  0.1 ,
        -0.48, -0.61])]

In [164]:
seq_collate([t[i] for i in [1, 2, 3, 4, 7]])[0].view(-1, 2)

RuntimeError: invalid argument 2: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Call .contiguous() before .view(). at /opt/conda/conda-bld/pytorch-cpu_1532578932944/work/aten/src/TH/generic/THTensor.cpp:237

In [145]:
np.linalg.norm(t.seq_list[10][0, 0, :], t.seq_list[10][1, 0, :])
    

/home/j.lappalainen/anaconda3/envs/dnnvsbrain/lib/python3.7/site-packages/numpy/linalg/linalg.py:2352: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  (ord in ('f', 'fro') and ndim == 2) or


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [25]:
import logging
import os
import math

import numpy as np

import torch
from torch.utils.data import Dataset

logger = logging.getLogger(__name__)


def seq_collate(data):
    (obs_seq_list, pred_seq_list, obs_seq_rel_list, pred_seq_rel_list,
     non_linear_ped_list, loss_mask_list) = zip(*data)

    _len = [len(seq) for seq in obs_seq_list]
    cum_start_idx = [0] + np.cumsum(_len).tolist()
    seq_start_end = [[start, end]
                     for start, end in zip(cum_start_idx, cum_start_idx[1:])]

    # Data format: batch, input_size, seq_len
    # LSTM input format: seq_len, batch, input_size
    obs_traj = torch.cat(obs_seq_list, dim=0).permute(2, 0, 1)
    pred_traj = torch.cat(pred_seq_list, dim=0).permute(2, 0, 1)
    obs_traj_rel = torch.cat(obs_seq_rel_list, dim=0).permute(2, 0, 1)
    pred_traj_rel = torch.cat(pred_seq_rel_list, dim=0).permute(2, 0, 1)
    non_linear_ped = torch.cat(non_linear_ped_list)
    loss_mask = torch.cat(loss_mask_list, dim=0)
    seq_start_end = torch.LongTensor(seq_start_end)
    out = [
        obs_traj, pred_traj, obs_traj_rel, pred_traj_rel, non_linear_ped,
        loss_mask, seq_start_end
    ]

    return tuple(out)


def read_file(_path, delim='\t'):
    data = []
    if delim == 'tab':
        delim = '\t'
    elif delim == 'space':
        delim = ' '
    with open(_path, 'r') as f:
        for line in f:
            line = line.strip().split(delim)
            line = [float(i) for i in line]
            data.append(line)
    return np.asarray(data)


def poly_fit(traj, traj_len, threshold):
    """
    Input:
    - traj: Numpy array of shape (2, traj_len)
    - traj_len: Len of trajectory
    - threshold: Minimum error to be considered for non linear traj
    Output:
    - int: 1 -> Non Linear 0-> Linear
    """
    t = np.linspace(0, traj_len - 1, traj_len)
    res_x = np.polyfit(t, traj[0, -traj_len:], 2, full=True)[1]
    res_y = np.polyfit(t, traj[1, -traj_len:], 2, full=True)[1]
    if res_x + res_y >= threshold:
        return 1.0
    else:
        return 0.0


class TrajectoryDataset(Dataset):
    """Dataloder for the Trajectory datasets"""
    def __init__(
        self, data_dir, obs_len=8, pred_len=12, skip=1, threshold=0.002,
        min_ped=1, delim='\t'
    ):
        """
        Args:
        - data_dir: Directory containing dataset files in the format
        <frame_id> <ped_id> <x> <y>
        - obs_len: Number of time-steps in input trajectories
        - pred_len: Number of time-steps in output trajectories
        - skip: Number of frames to skip while making the dataset
        - threshold: Minimum error to be considered for non linear traj
        when using a linear predictor
        - min_ped: Minimum number of pedestrians that should be in a seqeunce
        - delim: Delimiter in the dataset files
        """
        super(TrajectoryDataset, self).__init__()

        self.data_dir = data_dir
        self.obs_len = obs_len
        self.pred_len = pred_len
        self.skip = skip
        self.seq_len = self.obs_len + self.pred_len
        self.delim = delim

        all_files = os.listdir(self.data_dir)
        all_files = [os.path.join(self.data_dir, _path) for _path in all_files]
        num_peds_in_seq = []
        seq_list = []
        seq_list_rel = []
        loss_mask_list = []
        non_linear_ped = []
        for path in all_files:
            data = read_file(path, delim) # (frames, pid, x, y)
            frames = np.unique(data[:, 0]).tolist()
            frame_data = [data[np.where(data[:, 0]==x)] for x in frames]
            num_sequences = int(
                math.ceil((len(frames) - self.seq_len + 1) / self.skip))

            for idx in range(0, num_sequences * self.skip + 1, skip):
                curr_seq_data = np.concatenate(
                    frame_data[idx:idx + self.seq_len], axis=0)
                peds_in_curr_seq = np.unique(curr_seq_data[:, 1])
                curr_seq_rel = np.zeros((len(peds_in_curr_seq), 2,
                                         self.seq_len))
                curr_seq = np.zeros((len(peds_in_curr_seq), 2, self.seq_len)) # (num_peds, xy, seq_len)
                curr_loss_mask = np.zeros((len(peds_in_curr_seq),
                                           self.seq_len))
                num_peds_considered = 0 # initialize by 0
                _non_linear_ped = []
                for _, ped_id in enumerate(peds_in_curr_seq):
                    curr_ped_seq = curr_seq_data[curr_seq_data[:, 1] ==
                                                 ped_id, :]
                    curr_ped_seq = np.around(curr_ped_seq, decimals=4)
                    pad_front = frames.index(curr_ped_seq[0, 0]) - idx
                    pad_end = frames.index(curr_ped_seq[-1, 0]) - idx + 1
                    if pad_end - pad_front != self.seq_len:
                        #discard pedestrians that are less then seq_len in the scene
                        continue
                    curr_ped_seq = np.transpose(curr_ped_seq[:, 2:]) # (xy, seq_len)
                    # Make coordinates relative
                    rel_curr_ped_seq = np.zeros(curr_ped_seq.shape)
                    rel_curr_ped_seq[:, 1:] = \
                        curr_ped_seq[:, 1:] - curr_ped_seq[:, :-1] # kinda velocity
                    _idx = num_peds_considered
                    curr_seq[_idx, :, pad_front:pad_end] = curr_ped_seq # fill array with all pedestrians
                    curr_seq_rel[_idx, :, pad_front:pad_end] = rel_curr_ped_seq
                    # Linear vs Non-Linear Trajectory
                    _non_linear_ped.append(
                        poly_fit(curr_ped_seq, pred_len, threshold)) # 1 or 0 depending of the trajectory
                    curr_loss_mask[_idx, pad_front:pad_end] = 1 # ground truth for discriminator ?
                    num_peds_considered += 1
                
                if num_peds_considered > min_ped:
                    non_linear_ped += _non_linear_ped
                    num_peds_in_seq.append(num_peds_considered)
                    loss_mask_list.append(curr_loss_mask[:num_peds_considered])
                    seq_list.append(curr_seq[:num_peds_considered]) # (num_seq, num_peds, xy, seq_len)
                    seq_list_rel.append(curr_seq_rel[:num_peds_considered])    

        self.num_seq = len(seq_list)
        self.seq_list = np.concatenate(seq_list, axis=0)
        seq_list = self.seq_list
        seq_list_rel = np.concatenate(seq_list_rel, axis=0)
        loss_mask_list = np.concatenate(loss_mask_list, axis=0)
        non_linear_ped = np.asarray(non_linear_ped)

        # Convert numpy -> Torch Tensor
        self.obs_traj = torch.from_numpy(
            seq_list[:, :, :self.obs_len]).type(torch.float).permute(2, 0, 1)
        self.pred_traj = torch.from_numpy(
            seq_list[:, :, self.obs_len:]).type(torch.float).permute(2, 0, 1)
        self.obs_traj_rel = torch.from_numpy(
            seq_list_rel[:, :, :self.obs_len]).type(torch.float).permute(2, 0, 1)
        self.pred_traj_rel = torch.from_numpy(
            seq_list_rel[:, :, self.obs_len:]).type(torch.float).permute(2, 0, 1)
        self.loss_mask = torch.from_numpy(loss_mask_list).type(torch.float)
        self.non_linear_ped = torch.from_numpy(non_linear_ped).type(torch.float)
        cum_start_idx = [0] + np.cumsum(num_peds_in_seq).tolist()
        self.seq_start_end = [
            (start, end)
            for start, end in zip(cum_start_idx, cum_start_idx[1:])
        ]

    def __len__(self):
        return self.num_seq

    def __getitem__(self, index):
        start, end = self.seq_start_end[index]
        out = [
            self.obs_traj[:, start:end, :], self.pred_traj[:, start:end, :],
            self.obs_traj_rel[:, start:end, :], self.pred_traj_rel[:, start:end, :],
            self.non_linear_ped[start:end], self.loss_mask[start:end, :]
        ]
        return out


In [26]:
t = TrajectoryDataset('/mnt/Clouds/MapsGAN/data/eth/test')


In [30]:
t.loss_mask.shape

torch.Size([181, 20])

In [4]:
class Trajectories(Dataset):
    
    def __init__(self, data_dir, delim = '\t', seq_len = 8, pred_len = 12, max_peds = 32):
        super(Trajectories, self).__init__()
        self.data_dir = data_dir
        all_files = os.listdir(self.data_dir)
        self.all_files = [os.path.join(self.data_dir, _path) for _path in all_files]
        self.max_peds = max_peds
        
        for path in self.all_files:
            data = read_file(path, delim) # (frames, pid, x, y)
            frames = np.unique(data[:, 0]).tolist()
            self.frame_data = [data[np.where(data[:, 0]==x)] for x in frames]
            for idx, frame in enumerate(frame_data):
                # import pdb; pdb.set_trace();
                
                curr_seq_data = np.concatenate(frame_data[idx:idx + self.seq_len], axis=0)
                peds_in_curr_seq = np.unique(curr_seq_data[:, 1])
                
                curr_seq_rel = np.zeros((self.seq_len, 2, self.max_peds))
                curr_seq = np.zeros((self.seq_len, 2, self.max_peds)) # (num_peds, xy, seq_len)
                curr_loss_mask = np.zeros((self.seq_len, self.max_peds))
                num_peds_considered = 0 # initialize by 0
                
                for _, ped_id in enumerate(peds_in_curr_seq):
                    curr_ped_seq = curr_seq_data[curr_seq_data[:, 1] ==
                                                 ped_id, :]
                    curr_ped_seq = np.around(curr_ped_seq, decimals=4)
                    pad_front = frames.index(curr_ped_seq[0, 0]) - idx
                    pad_end = frames.index(curr_ped_seq[-1, 0]) - idx + 1
                    if pad_end - pad_front != self.seq_len:
                        #discard pedestrians that are less then seq_len in the scene
                        continue
                    curr_ped_seq = np.transpose(curr_ped_seq[:, 2:]) # (xy, seq_len)
                    # Make coordinates relative
                    rel_curr_ped_seq = np.zeros(curr_ped_seq.shape)
                    rel_curr_ped_seq[:, 1:] = \
                        curr_ped_seq[:, 1:] - curr_ped_seq[:, :-1] # kinda velocity
                    _idx = num_peds_considered
                    curr_seq[_idx, :, pad_front:pad_end] = curr_ped_seq # fill array with all pedestrians
                    curr_seq_rel[_idx, :, pad_front:pad_end] = rel_curr_ped_seq
                    # Linear vs Non-Linear Trajectory
                    _non_linear_ped.append(
                        poly_fit(curr_ped_seq, pred_len, threshold)) # 1 or 0 depending of the trajectory
                    curr_loss_mask[_idx, pad_front:pad_end] = 1 # ground truth for discriminator ?
                    num_peds_considered += 1
        
    def __getitem__(self):
           return NotImplemented

In [5]:
t = Trajectories('/mnt/Clouds/MapsGAN/data/eth/test')

In [13]:
np.unique(np.concatenate(t.frame_data[0:8], axis = 0)[:, 1])

array([1., 2., 3., 4., 5., 6.])

In [27]:
t.frame_data

[array([[780.  ,   1.  ,   8.46,   3.59]]),
 array([[790.  ,   1.  ,   9.57,   3.79]]),
 array([[800.  ,   1.  ,  10.67,   3.99],
        [800.  ,   2.  ,  13.64,   5.8 ]]),
 array([[810.  ,   1.  ,  11.73,   4.32],
        [810.  ,   2.  ,  12.09,   5.75]]),
 array([[820.  ,   1.  ,  12.81,   4.61],
        [820.  ,   2.  ,  11.37,   5.8 ]]),
 array([[830.  ,   2.  ,  10.31,   5.97],
        [830.  ,   3.  ,  12.49,   6.6 ]]),
 array([[840.  ,   2.  ,   9.57,   6.24],
        [840.  ,   3.  ,  11.94,   6.77]]),
 array([[850.  ,   2.  ,   8.73,   6.34],
        [850.  ,   3.  ,  11.03,   6.84],
        [850.  ,   4.  ,  -1.32,   5.11],
        [850.  ,   5.  ,  -1.48,   4.43],
        [850.  ,   6.  ,  11.84,   5.82]]),
 array([[ 8.600e+02,  2.000e+00,  7.940e+00,  6.500e+00],
        [ 8.600e+02,  3.000e+00,  1.021e+01,  6.810e+00],
        [ 8.600e+02,  4.000e+00, -3.300e-01,  5.060e+00],
        [ 8.600e+02,  5.000e+00, -4.900e-01,  4.380e+00],
        [ 8.600e+02,  6.000e+00,  1.09